This notebook is responsible for visualizing the data from the Github API, which should be provided in a serialized format.

It must be possible to run this notebook using jupyterlite.

(jupyterlite) We need to manually install dependencies for the front-end, even though they are also defined in the `requirements.txt`.

TODO: Fixed versions to prevent version-drift.

In [2]:
%pip install plotly 

Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install matplotlib

Note: you may need to restart the kernel to use updated packages.


In [4]:
%pip install ipywidgets

Note: you may need to restart the kernel to use updated packages.


In [41]:
import json
import datetime
import numpy as np
import plotly
import ipywidgets

In [43]:
with open('commit_dates.json') as f:
	commit_dates = json.load(f)
	
# Convert all the dates to datetime objects
commit_dates = [datetime.datetime.strptime(date, '%Y-%m-%d %H:%M:%S') for date in commit_dates]

In [70]:
# Plot a heatmap for the number of commits per day and hour of the week
def plot_heatmap(commit_dates):
	# Get the day and hour of each commit
	days = [date.weekday() for date in commit_dates]
	y_vals = [date.hour for date in commit_dates]
	
	# Create a 2D histogram of the day and hour
	heatmap, xedges, yedges = np.histogram2d(days, y_vals, bins=(7, 24))
	
	# Create a plotly figure
	fig = plotly.graph_objs.FigureWidget()
	
	# Add a heatmap trace
	fig.add_trace(plotly.graph_objs.Heatmap(
		z=heatmap,
		x=xedges,
		y=yedges,
		colorscale='oranges',
		colorbar=dict(
			title='Commits'
		),
		hoverinfo='z'
	))
	
	# Add axis labels
	fig.update_layout(
		xaxis=dict(
			title='Hour of the day',
			tickmode='array',
			tickvals=np.arange(0, 24, 2),
			ticktext=np.arange(0, 24, 2)
		),
		yaxis=dict(
			title='Day of the week',
			tickmode='array',
			tickvals=np.arange(0, 7),
			ticktext=['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
		)
	)
	
	return fig

plot_heatmap(commit_dates)

FigureWidget({
    'data': [{'colorbar': {'title': {'text': 'Commits'}},
              'colorscale': [[0.0, 'rgb(255,245,235)'], [0.125,
                             'rgb(254,230,206)'], [0.25, 'rgb(253,208,162)'],
                             [0.375, 'rgb(253,174,107)'], [0.5, 'rgb(253,141,60)'],
                             [0.625, 'rgb(241,105,19)'], [0.75, 'rgb(217,72,1)'],
                             [0.875, 'rgb(166,54,3)'], [1.0, 'rgb(127,39,4)']],
              'hoverinfo': 'z',
              'type': 'heatmap',
              'uid': 'a4fbb029-6d34-4a00-bf16-00a64350835e',
              'x': array([0.        , 0.85714286, 1.71428571, 2.57142857, 3.42857143, 4.28571429,
                          5.14285714, 6.        ]),
              'y': array([ 0.        ,  0.95833333,  1.91666667,  2.875     ,  3.83333333,
                           4.79166667,  5.75      ,  6.70833333,  7.66666667,  8.625     ,
                           9.58333333, 10.54166667, 11.5       , 12.45833333, 13.